Welcome to MX-Bluesky! This introduction is aimed at beamline scientists running from a beamline workstation. To confirm your environment is setup correctly,
run the below command to see if you get your expected beamline

In [ ]:
from dodal.utils import get_beamline_name

beamline = get_beamline_name("dev")
beamline = beamline.replace("-","_") # Convert ixx-n to ixx_n as this is the dodal format
print(f"Beamline is {beamline}")

Now let's import your beamline module and see what devices are available

In [21]:
import importlib

from dodal.utils import collect_factories

beamline = "i02_2"
module_name = f"dodal.beamlines.{beamline}"
beamline_module = importlib.import_module(module_name)

for device in collect_factories(beamline_module):
    print(device)

synchrotron
sample_motors


The Bluesky RunEngine must be used to interact with these devices. The RunEngine does a lot of things and is quite complicated. Some important benefits:
It can protect against using devices incorrectly, it allows us to pause and resume experiments, we can easily trace the sequence of events in experiments,
and we can customize it to do useful things like automatic plotting.

A Bluesky plan is the name for any experimental procedure. A plan can be anything from a simple motor movement to a complex data collection involving tens of devices. A plan is made up of plan stubs, which are the most simple atomic operations. The two most common plan stubs are called "abs_set" and "rd", and these are essentially just writing to and reading from the PV of a device. Here's what that would look like on a simulated motor:

In [ ]:
from bluesky import RunEngine
from bluesky import plan_stubs as bps
from ophyd_async.sim import SimMotor

RE = RunEngine() # Create the RunEngine. We only should be doing this once.
fake_motor = SimMotor() # Create the simulated motor

# Create the overall plan we want to run in the RunEngine
def my_plan():
    initial_position = yield from bps.rd(fake_motor.user_readback)
    print(f"Position of the motor before moving it is {initial_position}")
    #Now move the motor
    yield from bps.abs_set(fake_motor, 5, wait=True) #The code will wait here until the motor has finished moving. The units will be the same as that in EPICS
    final_position = yield from bps.rd(fake_motor.user_readback)
    print(f"Position of the motor after moving it is {final_position}")

# In Bluesky, when we actually run a plan, we must do it through the RunEngine like this
RE(my_plan())


The above code can look confusing, mainly because of the "yield from" syntax. We need this because Bluesky plans are python generators rather then regular functions.
You don't need to understand generators to use Bluesky, you just need to remember than when calling a plan stub, you need to do "yield from plan_name()". If you try and call it like a regular function, eg, just doing "plan_name()", this won't actually run, and the code may not error - it catches everyone out!
 
The above code ran instantly because the simulated motor has infinite velocity by default. We could set it to a realistic speed, then the code would take time to complete. Let's set the velocity using abs_set and try again:

In [ ]:
from time import time

RE(bps.abs_set(fake_motor, 0, wait=True)) # Move motor back to 0
RE(bps.abs_set(fake_motor.velocity, 2, wait=True)) #2 units per second
start_time = time()
RE(my_plan())
print(f"Movement took {round(time()-start_time,2)}s")

Note that the default acceleration of a fake motor is 0.5 units/s/s. To see what a device looks like and what signals are available to it, we recommend navigating the codebase using VSCode. With many devices including motors, the signals
of the dodal device will map very closely to the EPICS interface for that device. With a few exceptions, anything you are used to changing using EDM screens will also be doable inside a Bluesky plan.

Now try writing a simple Bluesky plan to move a real motor available using the above example code to help.